In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model  # TensorFlow is required for Keras to work
from PIL import Image, ImageOps  # Install pillow instead of PIL
import numpy as np

# A custom object to handle the 'groups' argument if it's causing issues during deserialization.
# This is a common workaround for DepthwiseConv2D errors due to version differences.
class FixedDepthwiseConv2D(tf.keras.layers.DepthwiseConv2D):
    def __init__(self, *args, **kwargs):
        kwargs.pop('groups', None) # Remove 'groups' if present
        super().__init__(*args, **kwargs)

    @classmethod
    def from_config(cls, config):
        config.pop('groups', None) # Remove 'groups' from config before passing to super's from_config
        return super().from_config(config)

# Disable scientific notation for clarity
np.set_printoptions(suppress=True)

# Load the model with the custom object
custom_objects = {'DepthwiseConv2D': FixedDepthwiseConv2D}
model = load_model("keras_model.h5", compile=False, custom_objects=custom_objects)

# Load the labels
class_names = open("labels.txt", "r").readlines()

# Create the array of the right shape to feed into the keras model
# The 'length' or number of images you can put into the array is
# determined by the first position in the shape tuple, in this case 1
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)

# Replace this with the path to your image
image = Image.open("/content/orange_01.jpg").convert("RGB")

# resizing the image to be at least 224x224 and then cropping from the center
size = (224, 224)
image = ImageOps.fit(image, size, Image.Resampling.LANCZOS)

# turn the image into a numpy array
image_array = np.asarray(image)

# Normalize the image
normalized_image_array = (image_array.astype(np.float32) / 127.5) - 1

# Load the image into the array
data[0] = normalized_image_array

# Predicts the model
prediction = model.predict(data)
index = np.argmax(prediction)
class_name = class_names[index]
confidence_score = prediction[0][index]

# Print prediction and confidence score
print("Class:", class_name[2:], end="")
print("Confidence Score:", confidence_score)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Class: 오렌지
Confidence Score: 0.9999615


In [ ]:
# TensorFlow(Keras) 로 모델을 로드하고 추론하기 위해 tensorflow를 import
import tensorflow as tf

# 저장된 Keras 모델(.h5)을 불러오기 위한 load_model 함수 import
from tensorflow.keras.models import load_model  # TensorFlow is required for Keras to work

# 이미지 로드/전처리를 위해 Pillow(PIL)에서 Image, ImageOps import
from PIL import Image, ImageOps  # Install pillow instead of PIL

# 배열 연산 및 모델 입력 텐서 구성을 위해 numpy import
import numpy as np


# -------------------------------
# (중요) Teachable Machine 모델 로드 시 DepthwiseConv2D 'groups' 호환성 문제 해결용 클래스
# -------------------------------
# 일부 TensorFlow/Keras 버전 차이 때문에, 모델 안의 DepthwiseConv2D 레이어가
# deserialization(역직렬화) 과정에서 'groups' 인자를 포함하고 있으면 에러가 날 수 있습니다.
# 그래서 'groups'가 들어오면 제거(pop)하고 부모 클래스 초기화로 넘기는 우회 방법을 사용합니다.
class FixedDepthwiseConv2D(tf.keras.layers.DepthwiseConv2D):
    def __init__(self, *args, **kwargs):
        # kwargs 안에 'groups'가 있으면 제거 (없으면 None 반환)
        kwargs.pop('groups', None)  # Remove 'groups' if present
        # 남은 인자/키워드 인자로 DepthwiseConv2D를 정상 초기화
        super().__init__(*args, **kwargs)

    @classmethod
    def from_config(cls, config):
        # config(dict) 안에 'groups'가 있으면 제거 (없으면 None 반환)
        config.pop('groups', None)  # Remove 'groups' from config before passing to super's from_config
        # 부모 클래스의 from_config로 레이어를 생성
        return super().from_config(config)


# -------------------------------
# 출력 가독성 설정
# -------------------------------
# numpy가 큰/작은 숫자를 1.23e-04 같은 과학적 표기법으로 출력하지 않도록 설정
np.set_printoptions(suppress=True)


# -------------------------------
# 모델 로드
# -------------------------------
# 커스텀 객체 매핑을 만들어, 모델 내부의 DepthwiseConv2D를 FixedDepthwiseConv2D로 대체 로딩하도록 지정
custom_objects = {'DepthwiseConv2D': FixedDepthwiseConv2D}

# (Colab 기준) Teachable Machine export 모델 파일 경로
# - 사용 환경에 따라 "/content/keras_model.h5"로 바꿔도 됩니다.
model_path = "keras_model.h5"

# 컴파일 없이(compile=False) 모델만 로드: 추론(predict)만 할 거라서 일반적으로 충분합니다.
model = load_model(model_path, compile=False, custom_objects=custom_objects)


# -------------------------------
# 라벨(클래스 이름) 로드
# -------------------------------
# (Colab 기준) labels 파일 경로
labels_path = "labels.txt"

# labels.txt의 각 줄을 리스트로 읽어옵니다. (각 원소는 한 줄 문자열)
# 예: ["0 사과\n", "1 오렌지\n", "2 포도\n"]
with open(labels_path, "r", encoding="utf-8") as f:
    class_names = f.readlines()


# -------------------------------
# 모델 입력 배열 준비
# -------------------------------
# Teachable Machine 이미지 모델은 기본적으로 (1, 224, 224, 3) 입력 형태를 기대합니다.
# - 1: 배치 크기(batch size) = 한 번에 1장 예측
# - 224,224: 이미지 크기
# - 3: RGB 채널
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)


# -------------------------------
# 예측할 이미지 로드 및 전처리
# -------------------------------
# 예측할 이미지 경로 (사용자 이미지로 교체)
image_path = "/content/orange_01.jpg"  # 예: "/content/내이미지.jpg"

# 이미지를 열고, 채널을 RGB로 강제 변환 (흑백/알파채널 등이 있어도 RGB로 맞춤)
image = Image.open(image_path).convert("RGB")

# 모델 입력 크기 (Teachable Machine 기본: 224x224)
size = (224, 224)

# ImageOps.fit:
# - 원본 이미지를 224x224 이상이 되도록 리사이즈한 뒤
# - 가운데(center)를 기준으로 224x224로 잘라(crop) 줍니다.
# - 비율을 유지하면서 중앙을 맞추기 때문에 왜곡이 적습니다.
# Image.Resampling.LANCZOS:
# - 고품질 리샘플링 방법(축소/확대 시 비교적 선명)
image = ImageOps.fit(image, size, Image.Resampling.LANCZOS)

# PIL 이미지를 numpy 배열(H, W, C) 형태로 변환
image_array = np.asarray(image)

# -------------------------------
# 정규화(Normalization)
# -------------------------------
# Teachable Machine Keras 이미지 모델은 보통 픽셀값을 [-1, 1] 범위로 정규화합니다.
# - 원본 픽셀: [0, 255]
# - (x / 127.5) - 1  => 0 -> -1, 127.5 -> 0, 255 -> 1
normalized_image_array = (image_array.astype(np.float32) / 127.5) - 1.0

# 배치의 첫 번째(0번째) 위치에 전처리된 이미지를 넣음
data[0] = normalized_image_array


# -------------------------------
# 추론(예측)
# -------------------------------
# model.predict(data):
# - 결과는 보통 (1, 클래스수) 형태의 확률/점수 배열
# - 예: [[0.02, 0.95, 0.03]]
prediction = model.predict(data)

# 가장 큰 값을 가지는 클래스 인덱스(argmax)를 찾음
index = np.argmax(prediction)

# labels.txt에서 해당 인덱스 줄을 가져옴 (예: "1 오렌지\n")
class_name_raw = class_names[index]

# 줄 끝 개행 제거 + 앞뒤 공백 제거
class_name_raw = class_name_raw.strip()

# "1 오렌지" 형태에서 사람이 보기 좋은 이름만 뽑고 싶으면 split 사용
# - 예: ["1", "오렌지"] -> "오렌지"
# - 라벨 포맷이 다르면(예: "오렌지"만 있는 파일) 이 부분을 조정해야 합니다.
parts = class_name_raw.split(" ", 1)         # 최대 한 번만 분리(앞 숫자와 나머지)
class_name = parts[1] if len(parts) > 1 else parts[0]  # 숫자+이름이면 이름만, 아니면 그대로 사용

# 해당 클래스의 confidence(점수/확률)를 가져옴
confidence_score = float(prediction[0][index])


# -------------------------------
# 결과 출력
# -------------------------------
print("Class:", class_name)
print("Confidence Score:", confidence_score)
